In [16]:
import os
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import Model, Input
from keras.applications.densenet import DenseNet201
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
import keras.backend as K
%matplotlib inline

In [17]:
X_train = np.load("128_128_train.npy")
y_train = np.load("train_labels.npy")
X_val = np.load("128_128_val.npy")
y_val = np.load("val_labels.npy")
X_test = np.load("128_128_test.npy")
y_test = np.load("test_labels.npy")

In [18]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [19]:
input_shape = X_val[0,:,:,:].shape
model_input = Input(shape=input_shape)

In [20]:
pre_trained_model = DenseNet201(input_shape=(192, 256, 3), include_top=False, weights="imagenet")

In [21]:
for layer in pre_trained_model.layers:
    print(layer.name)
    if hasattr(layer, 'moving_mean') and hasattr(layer, 'moving_variance'):
        layer.trainable = True
        K.eval(K.update(layer.moving_mean, K.zeros_like(layer.moving_mean)))
        K.eval(K.update(layer.moving_variance, K.zeros_like(layer.moving_variance)))
    else:
        layer.trainable = False

print(len(pre_trained_model.layers))

input_4
zero_padding2d_2
conv1/conv
conv1/bn
conv1/relu
zero_padding2d_3
pool1
conv2_block1_0_bn
conv2_block1_0_relu
conv2_block1_1_conv
conv2_block1_1_bn
conv2_block1_1_relu
conv2_block1_2_conv
conv2_block1_concat
conv2_block2_0_bn
conv2_block2_0_relu
conv2_block2_1_conv
conv2_block2_1_bn
conv2_block2_1_relu
conv2_block2_2_conv
conv2_block2_concat
conv2_block3_0_bn
conv2_block3_0_relu
conv2_block3_1_conv
conv2_block3_1_bn
conv2_block3_1_relu
conv2_block3_2_conv
conv2_block3_concat
conv2_block4_0_bn
conv2_block4_0_relu
conv2_block4_1_conv
conv2_block4_1_bn
conv2_block4_1_relu
conv2_block4_2_conv
conv2_block4_concat
conv2_block5_0_bn
conv2_block5_0_relu
conv2_block5_1_conv
conv2_block5_1_bn
conv2_block5_1_relu
conv2_block5_2_conv
conv2_block5_concat
conv2_block6_0_bn
conv2_block6_0_relu
conv2_block6_1_conv
conv2_block6_1_bn
conv2_block6_1_relu
conv2_block6_2_conv
conv2_block6_concat
pool2_bn
pool2_relu
pool2_conv
pool2_pool
conv3_block1_0_bn
conv3_block1_0_relu
conv3_block1_1_conv
conv3

In [22]:
last_layer = pre_trained_model.get_layer('relu')
print('last layer output shape:', last_layer.output_shape)
last_output = last_layer.output

last layer output shape: (None, 6, 8, 1920)


In [23]:
# Flatten the output layer to 1 dimension
x = layers.GlobalMaxPooling2D()(last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)
# Add a dropout rate of 0.7
x = layers.Dropout(0.5)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(7, activation='softmax')(x)

# Configure and compile the model

model = Model(pre_trained_model.input, x)
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [24]:
train_datagen = ImageDataGenerator(rotation_range=60, width_shift_range=0.2, height_shift_range=0.2,
                                   shear_range=0.2, zoom_range=0.2, fill_mode='nearest')

train_datagen.fit(X_train)

val_datagen = ImageDataGenerator()
val_datagen.fit(X_val)

In [25]:
batch_size = 32
epochs = 3
history = model.fit_generator(train_datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = val_datagen.flow(X_val, y_val),
                              verbose = 1, steps_per_epoch=(X_train.shape[0] // batch_size), 
                              validation_steps=(X_val.shape[0] // batch_size))

<ipython-input-25-140f2772e68d>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_datagen.flow(X_train,y_train, batch_size=batch_size),


Epoch 1/3
253/253 [==============================] - 1218s 5s/step - loss: 1.4029 - accuracy: 0.6267 - val_loss: 1.2677 - val_accuracy: 0.6696
Epoch 2/3
253/253 [==============================] - 1121s 4s/step - loss: 0.9098 - accuracy: 0.6866 - val_loss: 0.8146 - val_accuracy: 0.7087
Epoch 3/3
253/253 [==============================] - 1136s 4s/step - loss: 0.8140 - accuracy: 0.7102 - val_loss: 0.7414 - val_accuracy: 0.7199


In [26]:
pre_trained_model.layers[481].name

'conv5_block1_0_bn'

In [27]:
for layer in pre_trained_model.layers[481:]:
    layer.trainable = True

In [28]:
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])

In [29]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, 
                                            min_lr=0.0001, cooldown=2)

In [30]:
batch_size = 32
epochs = 20
history = model.fit_generator(train_datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = val_datagen.flow(X_val, y_val),
                              verbose = 1, steps_per_epoch=(X_train.shape[0] // batch_size),
                              validation_steps=(X_val.shape[0] // batch_size),
                              callbacks=[learning_rate_reduction])

<ipython-input-30-1caca08e3141>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_datagen.flow(X_train,y_train, batch_size=batch_size),


Epoch 1/20
253/253 [==============================] - 1136s 4s/step - loss: 0.7229 - acc: 0.7416 - val_loss: 0.6626 - val_acc: 0.7500 - lr: 1.0000e-04
Epoch 2/20
253/253 [==============================] - 1115s 4s/step - loss: 0.6108 - acc: 0.7786 - val_loss: 0.6058 - val_acc: 0.7746 - lr: 1.0000e-04
Epoch 3/20
253/253 [==============================] - 1132s 4s/step - loss: 0.5458 - acc: 0.8033 - val_loss: 0.5522 - val_acc: 0.7868 - lr: 1.0000e-04
Epoch 4/20
253/253 [==============================] - 1090s 4s/step - loss: 0.4851 - acc: 0.8204 - val_loss: 0.5591 - val_acc: 0.7935 - lr: 1.0000e-04
Epoch 5/20
253/253 [==============================] - 1007s 4s/step - loss: 0.4587 - acc: 0.8313 - val_loss: 0.5261 - val_acc: 0.8181 - lr: 1.0000e-04
Epoch 6/20
253/253 [==============================] - 1049s 4s/step - loss: 0.4268 - acc: 0.8437 - val_loss: 0.5163 - val_acc: 0.8192 - lr: 1.0000e-04
Epoch 7/20
253/253 [==============================] - 1058s 4s/step - loss: 0.4030 - acc: 0.84

In [33]:
model.save("model_densenet.h5")